In [10]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix,roc_curve,roc_auc_score,classification_report
import warnings 
warnings.filterwarnings('ignore')

In [11]:
df_train = pd.read_excel('Data_Train.xlsx')
df_train

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49 votes,1200
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30 votes,1500
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221 votes,800
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24 votes,800
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165 votes,300
...,...,...,...,...,...,...,...,...,...
12685,QUICK BITES,13228,"North Indian, Burger, Kebab","12noon – 12midnight (Mon, Tue, Wed, Thu, Sun)...",Hyderabad,Gachibowli,3.8,546 votes,500
12686,"CASUAL DINING,BAR",9686,"Goan, Continental","12noon – 1am (Mon-Fri),11am – 5pm, 7pm – 1am...",Mumbai,Bandra Kurla Complex,4.3,1214 votes,1800
12687,LOUNGE,11133,"Finger Food, Continental, Asian, Chinese",12noon – 12:30AM (Mon-Sun),Navi Mumbai,Vashi,4.0,608 votes,1300
12688,CASUAL DINING,6134,"North Indian, South Indian, Chinese, Street Food",6am – 10:45pm (Mon-Sun),Chennai,Maduravoyal,3.5,32 votes,400


In [12]:
df_train.shape

(12690, 9)

this data set contains 12690 rows  and 9 columns.This data set contains all the details related to restaurant food cost as well as the target variable “cost”.We will be predicting the cost of the food served by the restaurants across different cities in India.Hence the cost column is my label and all other remaining columns are my features. 

In [13]:
df_train.dtypes

TITLE            object
RESTAURANT_ID     int64
CUISINES         object
TIME             object
CITY             object
LOCALITY         object
RATING           object
VOTES            object
COST              int64
dtype: object

Through df_train.dtypes I observed that RESTAURANT_ID and COST has integer data type and TITLE, CUISINES, TIME, CITY, LOCALITY,RATING, VOTES these columns has object data type

In [14]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12690 entries, 0 to 12689
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TITLE          12690 non-null  object
 1   RESTAURANT_ID  12690 non-null  int64 
 2   CUISINES       12690 non-null  object
 3   TIME           12690 non-null  object
 4   CITY           12578 non-null  object
 5   LOCALITY       12592 non-null  object
 6   RATING         12688 non-null  object
 7   VOTES          11486 non-null  object
 8   COST           12690 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 892.4+ KB


Here also we can see as mentioned above out of 9 columns 2 columns has integer data type and 7 columns has object data type.Also TITLE, RESTAURANT_ID, CUISINES,TIME,COST these column Don't have any missing value.CITY, LOCALITY, RATING,VOTES in this column i can see NAN vlues present.

In [15]:
df_train.isna().sum()

TITLE               0
RESTAURANT_ID       0
CUISINES            0
TIME                0
CITY              112
LOCALITY           98
RATING              2
VOTES            1204
COST                0
dtype: int64

as mentioned above i can see CITY, LOCALITY, RATING,VOTES these columns have missing values.

In [16]:
df_train.nunique()

TITLE              113
RESTAURANT_ID    11892
CUISINES          4155
TIME              2689
CITY               359
LOCALITY          1416
RATING              32
VOTES             1847
COST                86
dtype: int64

here i can see out of 12690 records 'TITLE' has 113 unique values.'CITY' column has 359 unique values.'Rating' has 32 unique values.'cost' has 86 unique values,hence i belive these columns have categorical data  
'RESTAURANT_ID' and 'VOTES' can be considered as identifiers and continuous data, respectively, due to their nature and the way they are used in the dataset.
'CUISINES', 'TIME', and 'LOCALITY' are categorical data with a high cardinality due to the wide variety of possible values.As our label 'COST' has a categorical data and we have more than one feature to Analyse i can term it as Regression problem.

In [17]:
# let's check value count for each column
columns = 0  
while columns < 9:
    column_name = df_train.columns[columns]
    print(f"Value counts for {column_name}:")
    print(df_train[column_name].value_counts())
    print('\n')
    columns += 1


Value counts for TITLE:
QUICK BITES                 4218
CASUAL DINING               3652
None                        1003
CAFÉ                         607
DESSERT PARLOR               382
                            ... 
QUICK BITES,BAR                1
MEAT SHOP                      1
SWEET SHOP,CONFECTIONERY       1
BEVERAGE SHOP,FOOD COURT       1
BAR,MICROBREWERY               1
Name: TITLE, Length: 113, dtype: int64


Value counts for RESTAURANT_ID:
7264     8
4634     7
9910     6
11971    5
12747    5
        ..
9069     1
6573     1
7430     1
6381     1
6134     1
Name: RESTAURANT_ID, Length: 11892, dtype: int64


Value counts for CUISINES:
South Indian                                  532
North Indian                                  526
North Indian, Chinese                         501
Fast Food                                     286
Chinese                                       167
                                             ... 
Italian, Biryani, Fast Food              

In [19]:
df_train.sample(30)

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
706,QUICK BITES,3143,Chinese,11am – 10pm (Mon-Sun),Hyderabad,Banjara Hills,3.7,111 votes,200
8890,CASUAL DINING,11971,"South Indian, North Indian, Chinese, Street Fo...",7am – 11pm (Mon-Sun),Bangalore,Basaveshwara Nagar,3.8,74 votes,400
11934,"CASUAL DINING,CAFÉ",1419,"Chinese, Cafe, Fast Food",10am – 10pm (Mon-Sun),Bangalore,Nagarbhavi,4.5,1354 votes,700
4515,CASUAL DINING,5247,"Street Food, Chinese, South Indian, North Indi...",7am – 11:30pm (Mon-Sun),Mumbai,Santacruz East,3.8,463 votes,500
6004,CASUAL DINING,250,"Bengali, Chinese, Momos",12noon – 10:30pm (Mon-Sun),Secunderabad,Sainikpuri,3.5,58 votes,800
10270,None,5749,"North Indian, Chinese",24 Hours (Mon-Sun),Bangalore,Old Airport Road,NEW,NaN,300
11786,CASUAL DINING,13486,"Italian, Pizza",8:30am – 1am (Mon-Sun),Bandra West,Pali Hill,4.2,4583 votes,1600
8276,None,804,"Chinese, Momos",11:30am – 10:30pm (Mon-Sun),New Delhi,Rohini,3.7,148 votes,400
1192,QUICK BITES,7699,South Indian,8am – 11pm (Mon-Sun),Bangalore,Majestic,4.3,208 votes,150
6080,"DESSERT PARLOR,BEVERAGE SHOP",14556,"Desserts, Beverages",12noon – 1am (Mon-Sun),Mumbai,Chowpatty,4.2,269 votes,200


### cleaning and preprocessing the data

In [20]:
# as i can consider RESTAURANT_ID column as an identifier i will set this column as an index
df_train.set_index('RESTAURANT_ID',inplace=True)
df_train

,TITLE,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
RESTAURANT_ID,,,,,,,,
9438,CASUAL DINING,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49 votes,1200
13198,"CASUAL DINING,BAR","Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30 votes,1500
10915,CASUAL DINING,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221 votes,800
6346,QUICK BITES,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24 votes,800
15387,DESSERT PARLOR,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165 votes,300
...,...,...,...,...,...,...,...,...
13228,QUICK BITES,"North Indian, Burger, Kebab","12noon – 12midnight (Mon, Tue, Wed, Thu, Sun)...",Hyderabad,Gachibowli,3.8,546 votes,500
9686,"CASUAL DINING,BAR","Goan, Continental","12noon – 1am (Mon-Fri),11am – 5pm, 7pm – 1am...",Mumbai,Bandra Kurla Complex,4.3,1214 votes,1800
11133,LOUNGE,"Finger Food, Continental, Asian, Chinese",12noon – 12:30AM (Mon-Sun),Navi Mumbai,Vashi,4.0,608 votes,1300


In [21]:
# 'CITY', 'LOCALITY', 'RATING' these columns have relatively smaller numbers of missing values.
# we can handel these missing values using mode

for column in ['CITY', 'LOCALITY', 'RATING']:
    most_frequent = df_train[column].mode()[0]
    df_train[column].fillna(most_frequent, inplace=True)


In [22]:
# lets check missing values again
df_train.isna().sum()

TITLE          0
CUISINES       0
TIME           0
CITY           0
LOCALITY       0
RATING         0
VOTES       1204
COST           0
dtype: int64

There is no NAN values present any more in 'CITY', 'LOCALITY', 'RATING' these columns.

The 'VOTES' column has 1204 missing values, and since it's a significant number, we'll need to convert this column to a numerical format. hence this column contains numerical values followed by a text string ("votes"). we will have to use string manipulation to separate and extract numerical data

In [23]:

df_train['VOTES'] = df_train['VOTES'].astype(str)
df_train['VOTES'] = df_train['VOTES'].str.extract('(\d+)').astype(float)
# Fill missing values in 'VOTES' with 0 (assuming no votes means 0 votes)
df_train['VOTES'].fillna(0, inplace=True)



In [24]:
# lets check missing values again
df_train.isna().sum()

TITLE       0
CUISINES    0
TIME        0
CITY        0
LOCALITY    0
RATING      0
VOTES       0
COST        0
dtype: int64

In [25]:
# now there is no missing value present in any of the columns

In [26]:
df_train.dtypes

TITLE        object
CUISINES     object
TIME         object
CITY         object
LOCALITY     object
RATING       object
VOTES       float64
COST          int64
dtype: object

In [27]:
# lets look at 'RATING' column as in the DF it shows float data and with .dtypes it reflects as object data type.
df_train['RATING'].unique()

array(['3.6', '4.2', '3.8', '4.1', '4.0', '4.3', '3.9', '3.3', '3.4', '-',
       '4.5', '3.5', '4.4', '2.7', '3.7', '4.7', 'NEW', '3.1', '2.5',
       '4.6', '2.8', '3.0', '3.2', '2.6', '2.9', '4.9', '4.8', '2.4',
       '2.3', '2.0', '2.1', '2.2'], dtype=object)

In [28]:
# as we can see float numbers are Inside the single inverted comma,hence considerd as string. That's why they are reflecting as boject data type
# also there are 'NEW' AND '-' THESE  values are also present to convert htis column to float we will have to handelthis non numeric values as well

In [30]:
import numpy as np

# Replace 'NEW' and '-' with np.nan
df_train['RATING'] = df_train['RATING'].replace(['NEW', '-'], np.nan)

# Convert the column to float
df_train['RATING'] = df_train['RATING'].astype(float)

# Check the data type of the 'RATING' column now
df_train['RATING'].dtype

dtype('float64')

In [32]:
# lets check missing values again
df_train.isna().sum()

TITLE          0
CUISINES       0
TIME           0
CITY           0
LOCALITY       0
RATING      1202
VOTES          0
COST           0
dtype: int64

In [34]:
# as we replaced 'NEW' AND '-' THESE  non numeric values to NAN values thats why there are NAN values present in 'RATING' column again
# lets fix this issue
df_train['RATING'] = df_train['RATING'].fillna(df_train['RATING'].mode()[0])

In [35]:
# lets check missing values again
df_train.isna().sum()

TITLE       0
CUISINES    0
TIME        0
CITY        0
LOCALITY    0
RATING      0
VOTES       0
COST        0
dtype: int64

In [38]:
df_train.dtypes

TITLE        object
CUISINES     object
TIME         object
CITY         object
LOCALITY     object
RATING      float64
VOTES       float64
COST          int64
dtype: object

In [41]:
# i observed there are still 'TITLE','CUISINES','TIME','CITY','LOCALITY' these columns are showing object data type we have to convert it to numerical data type
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Encode categorical variables
for column in ['CITY', 'LOCALITY', 'TITLE', 'CUISINES']:
    df_train[column] = le.fit_transform(df_train[column])

print (df_train.dtypes)

TITLE         int64
CUISINES      int64
TIME         object
CITY          int64
LOCALITY      int64
RATING      float64
VOTES       float64
COST          int64
dtype: object


In [49]:
df_train.TIME.head()

RESTAURANT_ID
9438     11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)
13198                      6pm – 11pm (Mon-Sun)
10915       11am – 3:30pm, 7pm – 11pm (Mon-Sun)
6346                    11:30am – 1am (Mon-Sun)
15387                      11am – 1am (Mon-Sun)
Name: TIME, dtype: object

at this point i am not able to convert 'TIME'column. I believe that i should apply datetime function to convert it in datetime data type but i am not able to understand how to apply it here.